In [10]:
import numpy as np
import pandas as pd
from urllib.request import urlopen
from urllib import parse
from urllib.request import Request
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import json

In [15]:
client_id = "nrmyamgrqp"
client_pw = "IhbMAZssfW9TAXzmOUJaA8QRb32aAxih2onYVrPL"

api_url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query='

data = pd.read_excel('address.xlsx', names = ['지번주소'])

In [16]:
data.head()

,지번주소
0,대전 동구 하소동 361 옛터민속박물관
1,대전 동구 대성동 식장산
2,대전 동구 상소동 산1-1
3,대전 동구 가양동 8 남간정사
4,대전 동구 낭월동 산2-1


In [21]:
geo_coordi =[]
for add in data['지번주소']:
    add_urlenc = parse.quote(add)
    url = api_url + add_urlenc
    request = Request(url)
    request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
    request.add_header("X-NCP-APIGW-API-KEY",client_pw)
    try:
        response = urlopen(request)
    except HTTPError as e:
        print('HTTP Error')
        lat=None
        lon=None
        
    else:
        rescode = response.getcode()
        if rescode == 300:
            response_body = response.read().decode('utf-8')
            response_body = json.loads(response_body)
            if 'addresses' in response_body:
                lat=response_body['addresses'][0]['y']
                lon=response_body['addresses'][0]['x']
            else: 
                print('nothing')
                lat=None
                lon=None
                
        geo_coordi.append([lat, lon])
        
np_geo_coordi = np.array(geo_coordi)
np_geo_coordi = pd.DataFrame({"지번주소":data['지번주소'].values,
                             "위도": np_geo_coordi[:, 0],
                             "경도": np_geo_coordi[:, 1]})

In [23]:
np_geo_coordi.head()

,지번주소,위도,경도
0,대전 동구 하소동 361 옛터민속박물관,36.2151052,127.4402422
1,대전 동구 대성동 식장산,36.2151052,127.4402422
2,대전 동구 상소동 산1-1,36.2151052,127.4402422
3,대전 동구 가양동 8 남간정사,36.2151052,127.4402422
4,대전 동구 낭월동 산2-1,36.2151052,127.4402422


In [25]:
writer = pd.ExcelWriter('output.xlsx')
np_geo_coordi.to_excel(writer, sheet_name='Sheet1')
writer.save()